In [53]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [45]:
load_dotenv()

True

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", 
    task="conversational"
)

model = ChatHuggingFace(llm=llm)

In [47]:
# create a state
class LLMState(TypedDict):
    question: str
    answer: str

In [49]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [50]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [51]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

 The average distance from the Moon to the Earth is about 238,855 miles (384,400 kilometers). This distance varies because the elliptical shape of the Earth's orbit around the Sun causes the position of the Moon in its orbit around the Earth to change, resulting in a range of distances between approximately 225,623 miles (363,104 kilometers) at its closest (perigee) to 252,088 miles (405,696 kilometers) at its farthest (apogee).


In [55]:
model.invoke('How far is moon from the earth?').content

" The average distance from the Moon to the Earth is about 238,855 miles (384,400 kilometers). This distance varies throughout the lunar month because the Moon's elliptical orbit around Earth causes its position to be closer or farther from us. The closest distance, or Perigee, is approximately 225,623 miles (363,104 kilometers), and the farthest distance, or Apogee, is around 252,088 miles (405,696 kilometers)."